In [3]:
import pandas as pd
import json
import re
import nltk
import string
from bs4 import BeautifulSoup

file_path = 'Sarcasm_Headlines_Dataset.json'
data = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame(data)

print(df.head())
print(df.info())

FileNotFoundError: [Errno 2] No such file or directory: 'Sarcasm_Headlines_Dataset.json'

In [ ]:
del df['article_link'] # column not used

print(df.isnull().sum()) 
# df,dropna(inplcae=True) # if None value exists

df.drop_duplicates(subset=['headline'], inplace=True) # remove the repeated data

In [ ]:
nltk.download('stopwords')
stop = set(stopwords.words('english')) # frequent english stopwords (the, is, and ...)
punctuation = list(string.punctuation) # punctuation (, . ! ? ...)
stop.update(punctuation) # add the punctuation to the stopwords set

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text() # remove the HTML label (<p> <a> <xxx>...)

def remove_urls(text):
    return re.sub(r'http\S+', '', text) # remobe the urls (https://test.com ...)

def remove_between_square_brackets(text): 
    return re.sub('\[[^]]*\]', '', text) # remove the '[content]'

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text) # remove the specific words in stopwords set(stopwords+punctuation)

abbreviations = {
    r"\bI'm\b": "I am",
    r"\bit's\b": "it is",
    r"\bcan't\b": "cannot",
    r"\bdon't\b": "do not",
    r"\bwon't\b": "will not",
    r"\bI'm\b": "I am",
    r"\bI've\b": "I have",
    r"\blet's\b": "let us",
    r"\bthey're\b": "they are",
    r"\bhe's\b": "he is",
    r"\bshe's\b": "she is",
    r"\bwe're\b": "we are",
    r"\byou're\b": "you are",
}

def expand_contractions(text):
    for abbr, full_form in abbreviations.items():
        text = re.sub(abbr, full_form, text)
    return text # replace the abbreviation to full-format

def remove_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def denoise_text(text):
    text = strip_html(text)
    text = remove_urls(text)
    text = text.lower() # lower all the characters
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text = expand_contractions(text)
    text = remove_spaces(text)
    return text

df['headline'] = df['headline'].apply(denoise_text) #Apply function on review column

In [ ]:
print(df.iloc[106]['headline'])